In [22]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

In [23]:
races = pd.read_csv(path+'data/races.csv')

races.head()

,season,round,circuit_id,country,lat,long,date,url
0,2020,1,red_bull_ring,Austria,47.2197,14.76470,2020-07-05,http://en.wikipedia.org/wiki/2020_Austrian_Gra...
1,2020,2,red_bull_ring,Austria,47.2197,14.76470,2020-07-12,http://en.wikipedia.org/wiki/2020_Styrian_Gran...
2,2020,3,hungaroring,Hungary,47.5789,19.24860,2020-07-19,http://en.wikipedia.org/wiki/2020_Hungarian_Gr...
3,2020,4,silverstone,UK,52.0786,-1.01694,2020-08-02,http://en.wikipedia.org/wiki/2020_British_Gran...
4,2020,5,silverstone,UK,52.0786,-1.01694,2020-08-09,http://en.wikipedia.org/wiki/70th_Anniversary_...


In [24]:
weather = races.iloc[:,[0,1,2]]

In [25]:
weather

,season,round,circuit_id
0,2020,1,red_bull_ring
1,2020,2,red_bull_ring
2,2020,3,hungaroring
3,2020,4,silverstone
4,2020,5,silverstone
5,2020,6,catalunya
6,2020,7,spa
7,2020,8,monza
8,2020,9,mugello
9,2020,10,sochi


In [26]:
info = []

for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()
                        
                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima) 
                                
    except:
        info.append('not found')

In [27]:
weather['weather'] = info

weather.head()

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_36995/2711338389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['weather'] = info


,season,round,circuit_id,weather
0,2020,1,red_bull_ring,not found
1,2020,2,red_bull_ring,not found
2,2020,3,hungaroring,not found
3,2020,4,silverstone,not found
4,2020,5,silverstone,not found


In [19]:
weather_dict = {'warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'cold': ['cold', 'fresh', 'chilly', 'cool'],
               'dry': ['dry', 'asciutto'],
               'wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

In [20]:
weather_df = pd.DataFrame(columns = weather_dict.keys())
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: True if any(i in weather_dict[col] for i in x.lower().split()) else False)

# weather_df['dry'] = weather_df.wet.map(lambda x: True if x == False else False)   
weather_df.head()

,warm,cold,dry,wet,cloudy
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False


In [21]:
weather_info = pd.concat([weather, weather_df], axis = 1)

# weather_info.to_csv(path+'data/weather.csv', index = False)
weather_info.query('wet == False')
# weather_info.head()

,season,round,circuit_id,weather,warm,cold,dry,wet,cloudy
0,2020,1,red_bull_ring,not found,False,False,False,False,False
1,2020,2,red_bull_ring,not found,False,False,False,False,False
2,2020,3,hungaroring,not found,False,False,False,False,False
3,2020,4,silverstone,not found,False,False,False,False,False
4,2020,5,silverstone,not found,False,False,False,False,False
5,2020,6,catalunya,not found,False,False,False,False,False
6,2020,7,spa,not found,False,False,False,False,False
7,2020,8,monza,not found,False,False,False,False,False
8,2020,9,mugello,not found,False,False,False,False,False
9,2020,10,sochi,not found,False,False,False,False,False
